# <font size = "6"> **Projet Pizzo Topping - Morel, Guinot et Mssellati**

Dans ce projet nous allons tenter de prédire les toppings sur des images de pizza. Nous nous sommes concentré sur la base des pizzas générées artificiellement. 

Nous avons tenté et adapté notre approche au fur et à mesure du projet. 
Nous vous les présentons dans un ordre à complexité croissante. 


In [ ]:
##Bibliotheque utiles dans tous le notebook
import pandas as pd
import numpy as np
import os
import time
import matplotlib.pyplot as plt
from PIL import Image

from tqdm import tqdm

import seaborn as sns
sns.set_style('darkgrid')

#import shutil

import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

# scikitlearn
from sklearn.metrics import confusion_matrix, classification_report, f1_score, ConfusionMatrixDisplay, multilabel_confusion_matrix
from sklearn.model_selection import train_test_split

# Pytorch
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision
import torchvision.transforms as transforms
from torchvision import models
from torch.optim import lr_scheduler
import torch.nn.functional as F
import torch.nn as nn

#!pip install torchmetrics
#from torchmetrics.classification import MultilabelF1Score, MultilabelAccuracy

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


if (torch.cuda.is_available()):
  !nvidia-smi

## <font size = "7"> CNN simple </font>





## <font size = "7"> Feature extraction

Dans cette partie nous allons étudier l'efficacité de différentes méthodes d'apprentissage en sortit d'un réseau de neurones pre entrainé. 

La première approche a été de classifier la sortie de ResNet avec des méthodes de classification supervisé vues en cours.

La deuxième consiste à entraîner un MLP avec la sortie d'un CNN déjà entrainé. 

Noter qu'ici nous ne touchons en rien aux poids du réseau initial, seule la sortie est traitée.

### <font size = "6"> ResNext + Classifier

Un Resnet est un réseau qui va interconnecter des neurones jusqu'à plus d'une couche. Il est composé de bloc résiduel les uns à la suite des autres. Cela permet de réduire l'explosion ou la chute du gradient durant l'optimisation. Ce phénomène apparaît lorsqu'il y a beaucoup de couches. Nous avons donc suivi dans un premier temps les indications du sujet et traité la sortie du Resnet. En particulier, c'est un ResNext 50 qui est utilisé, utilisant également à de l'inception et de la profondeur.

Dans cette partie, nous allons tester différentes méthodes de classification en sortie du ResNext. ResNext reste assez simple et léger pour évaluer l'ensemble du data set.

In [ ]:
#Bliotheque necessaire à la classification 
from sklearn.multioutput import MultiOutputClassifier, ClassifierChain
from sklearn.svm import LinearSVC
from sklearn.linear_model import Perceptron
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, StackingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import multilabel_confusion_matrix, ConfusionMatrixDisplay

In [ ]:
def True_ratio(y_val,y_val_pred):
  ratio_list=[]
  for i in range(y_val.shape[1]): 
    nb_of_true_overall=0
    nb_of_true=0
    ratio=0
    for j in range(y_val.shape[0]) :
      if y_val[j,i]==1 :
        nb_of_true_overall=nb_of_true_overall+1
        if y_val_pred[j,i]==y_val[j,i] :
          nb_of_true=nb_of_true+1
    if nb_of_true_overall!=0 :
      ratio=nb_of_true/nb_of_true_overall
      ratio_list.append(ratio)
  return ratio_list

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(feat_double, y_all_double, test_size=0.2, random_state=123)

#### SVM
Ici, nous testons un SVM lineair.

Dans cette partie et dans la suite, il est important de préciser à SKLearn que les outputs sont multiples.

In [ ]:
multiclass_SVM = MultiOutputClassifier(LinearSVC(random_state=42, C=2))

In [ ]:
#Training
multiclass_SVM= multiclass_SVM.fit(X_train, y_train)

In [ ]:
#Predict 
y_val_pred = multiclass_SVM.predict(X_val)

Ici se pose la question de la représentions de l'efficacité des classifieurs.
 
Nous avons d'abord pensé à des matrices de confusion pour chacune des classes. 

In [ ]:
# Confusion Matrix multiclass
matrices = multilabel_confusion_matrix(y_val, y_val_pred)
nlabels = 10
for k in range(nlabels):
  cmd = ConfusionMatrixDisplay(matrices[k], display_labels=np.unique(y_test)).plot()
  plt.title('Confusion Matrix for label {}'.format(k+1))
  plt.show()

Il est assez compliqué d'interpréter les résultats. 

En effet, dans la majorité des cas, pour un topping donné, il ne sera pas là. C'est la raison pour laquelle le cadrant en haut à gauche est rempli.
De plus, il est assez difficile de voir autant de matrice d'un coup. 
Les matrices ne sont pas forcément adaptées à l'analyse de méthodes pour notre problème. 

Nous nous sommes dirigés naturellement vers un pourcentage réussite par classe. 

In [ ]:
# Bar charts for average predictios
perf_label_val = np.zeros(nlabels)
for k in range(len(y_val_pred)):
  perf_label_val = perf_label_val + (y_val[k] == y_val_pred[k])

res = perf_label_val/y_val_pred.shape[0]*100

print(res)

plt.figure()
plt.bar(np.linspace(0,10,10), perf_label_val/y_val_pred.shape[0]*100)

Ici, les scores peuvent paraitre bons. Mais comme souligne déjà plus, ce que nous voulons principalement, c'est savoir : pour un topping donné, quand est-ce que le modèle a bien prédit. 

Nous avons donc mis en place cette métrique : True_ratio. 
Elle peut paraitre trop forte de prime abord, mais elle a le mérite de briller lorsque les résultats sont convaincants.

In [ ]:
# Bar charts for when true predictions
Ratios_SVM = True_ratio(y_val,y_val_pred)
print(Ratios_SVM)
plt.figure()
plt.bar(np.linspace(0,10,10),Ratios_SVM)
print('Overall : ', np.array(Ratios_SVM).mean() ) # moyenne des scores

Nous pouvons donc voir ici que le SVM ne produit pas des scores très élevés.

Nous avions également testé un SVM non linéaire, mais le temps d'entrainement et de prédiction était bien trop élevé pour un score trop bas. 

#### KNN 

Nous essayons egalment les K voisins :

In [ ]:
multiclass_KNN = MultiOutputClassifier(KNeighborsClassifier(n_neighbors=7, weights='uniform', algorithm='auto', leaf_size=30, p=2, 
                             metric='minkowski', metric_params=None, n_jobs=None))

Apres quelques tests, 7 est un bon compromis perfomance resultats

In [ ]:
#Predict (no training for Knn)
y_val_pred = multiclass_KNN.predict(X_val)

In [ ]:
# Bar charts for when true predictions
Ratios_knn = True_ratio(y_val,y_val_pred)
print(Ratios_knn)
plt.figure()
plt.bar(np.linspace(0,10,10),Ratios_knn)
print('Overall : ', np.array(Ratios_knn).mean() ) # moyenne des scores

#### Random Forest



In [ ]:
multiclass_RandomFor = MultiOutputClassifier(RandomForestClassifier(n_estimators=50, criterion='gini', max_depth=None,
                               min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0,
                               max_features='sqrt', max_leaf_nodes=None, min_impurity_decrease=0.0, bootstrap=True, 
                               oob_score=False, n_jobs=None, random_state=None, verbose=0, warm_start=False, class_weight=None, 
                               ccp_alpha=0.0, max_samples=None))


In [ ]:
#Training
multiclass_RandomFor = multiclass_RandomFor.fit(X_train, y_train)

In [ ]:
#Predict 
y_val_pred = multiclass_RandomFor.predict(X_val)

In [ ]:
# Bar charts for when true predictions
Ratios_for = True_ratio(y_val,y_val_pred)
print(Ratios_for)
plt.figure()
plt.bar(np.linspace(0,10,10),Ratios_for)
print('Overall : ', np.array(Ratios_for).mean() ) # moyenne des scores

Comme on peut le voir Random forrest ne fournit pas des resultats satisfaisant.

#### Perceptron

In [ ]:
multiclass_Perceptron = MultiOutputClassifier(Perceptron(penalty = 'elasticnet',verbose=1))


Apres quelques tests, elasticnet avait l'air de fonctionner le mieux .

In [ ]:
multiclass_Perceptron = multiclass_Perceptron.fit(X_train, y_train)

In [ ]:
#Predict 
y_val_pred = multiclass_Perceptron.predict(X_val)

In [ ]:
# Bar charts for when true predictions
Ratios_Per = True_ratio(y_val,y_val_pred)
print(Ratios_Per)
plt.figure()
plt.bar(np.linspace(0,10,10),Ratios_Per)
print('Overall : ', np.array(Ratios_Per).mean() ) # moyenne des scores

Le perceptron fournis des résultats convaincant par rapport à d'autre classifieur. C'est un équivalent à un MLP à une couche. Ce qui nous a poussé à greffer un MLP un peu plus développé à la sortie du réseau pré entrainé sur des images. 

### <font size = "6"> Pre-trained CNN + MLP


#### ResNet + MLP

#### VGG + MLP

#### AlexNet + MLP

## <font size = "7"> Fine Tuning



## <font size = "6"> **Conclusion**

